In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [ ]:
df=pd.read_csv('ch07_mobile_price.csv')
model=RandomForestClassifier(random_state=1234)

X=df.drop('price_range',axis=1)
y=df['price_range']

# filter method

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import numpy as np

test = SelectKBest(score_func=chi2, k=X.shape[1])
fit = test.fit(X, y)# summarize evaluation scores
print(np.round(fit.scores_, 3))
f_order= np.argsort(-fit.scores_)  # sort index by decreasing order
sorted_columns= df.columns[f_order]

In [ ]:
for i in range(1, X.shape[1]+1):
    fs = sorted_columns[0:i]
    df_X_selected= X[fs]
    scores = cross_val_score(model, df_X_selected, y, cv=10)
    print(fs.tolist())
    print(np.round(scores.mean(), 4))


# Backward elimination

> 한개씩 feature 를 빼면서 선택한다


In [ ]:
from sklearn.feature_selection import RFE
import pandas as pd
from sklearn.model_selection import cross_validate,cross_val_score


for i in range(1,len(X.columns)):
    print(i)
    rfe= RFE(model, n_features_to_select=i)
    fit = rfe.fit(X, y)
    print("Num Features: %d" % fit.n_features_)
    fs = X.columns[fit.support_].tolist()   # selected features
    print("Selected Features: %s" % fs)
    scores = cross_val_score(model, X[fs], y, cv=10)
    print("Acc: "+str(scores.mean()))


#정확도가 가장 좋은 4개 피처를 선택

# Forward selection
> 한개씩 추가해서 

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFE
import pandas as pd
from sklearn.model_selection import cross_validate,cross_val_score

#forward selection

sfs1 = SFS(model, k_features=len(X.columns),          # number of features
            verbose=2,scoring='accuracy',cv=10)
sfs1 = sfs1.fit(X, y)
print(sfs1.subsets_)             # selection process
print(sfs1.k_feature_idx_)       # selected feature index  
print(sfs1.k_feature_names_)


scores = cross_val_score(model, X[list(sfs1.subsets_[5]['feature_names'])], y, cv=10)

print("Acc: "+str(scores.mean()))



In [87]:
from sklearn.ensemble import StackingClassifier
import warnings

warnings.filterwarnings('ignore')


estimators = [('rf', RandomForestClassifier()),
              ('svr', make_pipeline(StandardScaler(),LinearSVC())),
              ('dt', DecisionTreeClassifier())]
# define model
model_1 = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=500))
scores_1 = cross_val_score(model_1, X, y, cv=10)
print(np.mean(scores_1))

0.9640000000000001
